In [4]:
!pip install google-cloud-storage
!pip install google-cloud-bigquery


In [1]:
from google.colab import auth
auth.authenticate_user()

# Set your GCP project
!gcloud config set project prj-hdfc-ergo-genai


Updated property [core/project].


In [2]:
from google.cloud import storage, bigquery

# Initialize GCS and BigQuery clients
storage_client = storage.Client()
bigquery_client = bigquery.Client()


In [4]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# List files in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)


comp-analysis-new-data/
comp-bq-segmentwise/
comp-bq-segmentwise/excel-sheets/
comp-bq-segmentwise/excel-sheets/segment_April_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_April_2024.xlsx
comp-bq-segmentwise/excel-sheets/segment_August_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_December_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_February_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_February_2024.xlsx
comp-bq-segmentwise/excel-sheets/segment_January_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_January_2024.xlsx
comp-bq-segmentwise/excel-sheets/segment_July_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_June_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_March_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_March_2024.xlsx
comp-bq-segmentwise/excel-sheets/segment_May_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_May_2024.xlsx
comp-bq-segmentwise/excel-sheets/segment_November_2023.xlsx
comp-bq-segmentwise/excel-sheets/segment_October_2023.x

In [5]:
from google.cloud import storage
import pandas as pd

# Initialize the GCS client
storage_client = storage.Client()

# Define the bucket name and the folder path
bucket_name = 'hdfc-segmetwise-data'
folder_path = 'comp-segmentwise/'  # Folder path inside the bucket

# Access the bucket
bucket = storage_client.get_bucket(bucket_name)

# List all files in the specified folder and filter CSV files
blobs = bucket.list_blobs(prefix=folder_path)

# Loop over the files and process only CSV files
for blob in blobs:
    if blob.name.endswith('.csv'):  # Check if it's a CSV file
        print(f"Processing file: {blob.name}")

        # Download the CSV file to a local temporary file
        local_file_name = blob.name.split('/')[-1]  # Extract the file name
        blob.download_to_filename(local_file_name)

        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(local_file_name)

        # Do something with the DataFrame (like printing the first few rows)
        print(df.head())  # Display the first few rows of the CSV

        # Optionally, you can load the DataFrame into BigQuery or perform other operations
        # For example:
        # bq_client.load_table_from_dataframe(df, 'your-project.your_dataset.your_table')


Processing file: comp-segmentwise/segmentwise-report_april_2022.xlsx - Segmentwise Report.csv
  GROSS DIRECT PREMIUM INCOME UNDERWRITTEN BY NON-LIFE INSURERS WITHIN INDIA  (SEGMENT WISE) : FOR THE PERIOD UPTO April 2022 (PROVISIONAL & UNAUDITED ) IN FY 2022-23  (Rs. In Crs.)  \
0                                                NaN                                                                                                                                    
1                                   General Insurers                                                                                                                                    
2                         Acko General Insurance Ltd                                                                                                                                    
3                                      Previous Year                                                                                                                  

In [9]:
from google.cloud import bigquery
import pandas as pd
from google.cloud import storage
import os

# Initialize BigQuery and GCS clients
bq_client = bigquery.Client()
storage_client = storage.Client()

# Define the GCS bucket name
bucket_name = 'hdfc-segmetwise-data'  # Replace with your actual bucket name
bucket = storage_client.bucket(bucket_name)

# Define your BigQuery project and dataset
project_id = 'prj-hdfc-ergo-genai'  # Replace with your project ID
dataset_id = 'prj-hdfc-ergo-genai.hdfc_ergo_segment'  # Replace with your dataset ID

# Function to process and ingest each CSV into its own table
def process_csv_file(blob):
    try:
        if blob.name.endswith('.csv'):
            print(f"Processing file: {blob.name}")

            # Extract the filename (e.g., 'segmentwise-report_april_2022.csv')
            local_file_name = blob.name.split('/')[-1]

            # Download the CSV file to a local path
            blob.download_to_filename(local_file_name)

            # Load the CSV file into a pandas DataFrame
            df = pd.read_csv(local_file_name)

            # Extract month and year from the file name for table creation
            # Assuming the file name format is consistent like "segmentwise-report_month_year.csv"
            file_parts = local_file_name.replace('.csv', '').replace('-', '_').split('_')
            if len(file_parts) >= 3:
                month = file_parts[-2]  # Get month (e.g., 'april')
                year = file_parts[-1]  # Get year (e.g., '2022')
                table_name = f"segmentwise_{month}_{year}".lower()  # Create table name

                # Clean the table name (remove invalid characters)
                table_name = table_name.replace('-', '_').replace(' ', '_').lower()

                # Create the full table reference
                table_ref = f"{project_id}.{dataset_id}.{table_name}"

                # Set up job configuration for BigQuery
                job_config = bigquery.LoadJobConfig(
                    source_format=bigquery.SourceFormat.CSV,
                    skip_leading_rows=1,
                    autodetect=True
                )

                # Ingest the DataFrame into BigQuery
                print(f"Uploading data to table: {table_ref}")
                job = bq_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
                job.result()  # Wait for the job to complete
                print(f"Data from {local_file_name} successfully loaded to {table_ref}")

            else:
                print(f"Unexpected file naming format for {local_file_name}")

            # Clean up the local file
            os.remove(local_file_name)

    except Exception as e:
        print(f"Error processing {blob.name}: {str(e)}")

# List all blobs in the bucket
blobs = list(bucket.list_blobs(prefix='comp-segmentwise/'))  # Prefix to focus on your CSV directory

# Process each CSV file
for blob in blobs:
    process_csv_file(blob)

print("All CSV files have been processed.")


Processing file: comp-segmentwise/segmentwise-report_april_2022.xlsx - Segmentwise Report.csv
Uploading data to table: prj-hdfc-ergo-genai.prj-hdfc-ergo-genai.hdfc_ergo_segment.segmentwise_2022.xlsx___segmentwise_report
Error processing comp-segmentwise/segmentwise-report_april_2022.xlsx - Segmentwise Report.csv: table_id must be a fully-qualified ID in standard SQL format, e.g., "project.dataset.table_id", got prj-hdfc-ergo-genai.prj-hdfc-ergo-genai.hdfc_ergo_segment.segmentwise_2022.xlsx___segmentwise_report
Processing file: comp-segmentwise/segmentwise-report_april_2023.xlsx - Segmentwise Report.csv
Uploading data to table: prj-hdfc-ergo-genai.prj-hdfc-ergo-genai.hdfc_ergo_segment.segmentwise_2023.xlsx___segmentwise_report
Error processing comp-segmentwise/segmentwise-report_april_2023.xlsx - Segmentwise Report.csv: table_id must be a fully-qualified ID in standard SQL format, e.g., "project.dataset.table_id", got prj-hdfc-ergo-genai.prj-hdfc-ergo-genai.hdfc_ergo_segment.segmentwise